# **Librerías necesarias**

In [ ]:
install.packages("cluster")
install.packages("ggplot2")
library(cluster)
library(ggplot2)

Cargamos los datos y verificamos los tipos y su estructura

In [ ]:
mall_customers <- read.csv("mallCustomers.csv")
str(mall_customers)
head(mall_customers)
summary(mall_customers)

# **Agrupamiento**

### **k-means**

Primero, calcularemos el número óptimo de centroides a utilizar mediante el método del codo

In [ ]:

set.seed(123)
wcss <- vector()
for (i in 1:10) {
  kmeans_model <- kmeans(mall_customers[, c("Age", "Annual.Income", "Spending.Score")], centers = i, nstart = 10)
  wcss[i] <- kmeans_model$tot.withinss
}

Graficamos los resultados

In [ ]:
plot(1:10, wcss, type = 'b', main = "Método del Codo", xlab = "Número de Clústeres (k)", ylab = "WCSS")

Determinamos que el número óptimo de centroides es 5, por lo tanto, aplicamos el algoritmo con dicho número, usando todas las variables excepto el género

In [ ]:
kmeans_model <- kmeans(mall_customers[, c("Age", "Annual.Income", "Spending.Score")], centers = 5, nstart = 10)
mall_customers$cluster <- kmeans_model$cluster

Calculamos la información retenida para verificar la calidad del modelo

In [ ]:
centroids <- kmeans_model$centers
distances_to_centroid <- as.matrix(dist(mall_customers[, c("Age", "Annual.Income", "Spending.Score")], method = "euclidean"))
withinss <- rep(0, 5)
for (i in 1:5) {
  withinss[i] <- sum((distances_to_centroid[, i])^2)
}
tot_withinss <- sum(withinss)
totss <- sum((distances_to_centroid)^2)
betweenss <- totss - tot_withinss
information <- betweenss / totss
information

La información retenida es muy alta, lo que indica que es una buena agrupación

Graficamos los resultados

In [ ]:
ggplot(mall_customers, aes(x = Annual.Income, y = Spending.Score, color = factor(cluster))) +
  geom_point() +
  geom_point(data = kmeans_model$centers, aes(x = Annual.Income, y = Spending.Score), color = 'black', size = 3, shape = 16) +
  labs(title = "Segmentación de clientes", x = "Ingresos Anuales", y = "Puntuación de Gasto") +
  theme_minimal()

Observamos los resultados de forma numérica

In [ ]:

table(mall_customers$cluster)
prop.table(table(mall_customers$cluster))


In [ ]:
cluster_stats <- aggregate(. ~ cluster, data = mall_customers[, c("Annual.Income", "Spending.Score", "cluster")], FUN = function(x) c(mean = mean(x), sd = sd(x), min = min(x), max = max(x)))
cluster_stats

### **Dendrogramas Jerárquicos**

##### **Single**

Creamos el dendrograma usando el método single

In [ ]:
dendrogram_single <- hclust(dist(mall_customers[, c("Age", "Annual.Income", "Spending.Score")]), method = "single")

Graficamos el resultado

In [ ]:
plot(dendrogram_single, main = "Single", xlab = "", ylab = "Distancia")

##### **Centroid**

Creamos el dendrograma usando el método centroid

In [ ]:
dendrogram_centroid <- hclust(dist(mall_customers[, c("Age", "Annual.Income", "Spending.Score")]), method = "centroid")

Graficamos el resultado

In [ ]:
plot(dendrogram_centroid, main = "Centroid", xlab = "", ylab = "Distancia")

##### **Ward**

Creamos el dendrograma usando el método ward

In [ ]:
dendrogram_ward <- hclust(dist(mall_customers[, c("Age", "Annual.Income", "Spending.Score")]), method = "ward.D")

Graficamos el resultado

In [ ]:
plot(dendrogram_ward, main = "Ward", xlab = "", ylab = "Distancia")

##### **Evaluación de los dendrogramas**

Comparamos los dendrogramas y nos quedamos con el que preserva mejor las distancias originales mediante el coeficiente de correlación de Cophenetic

In [ ]:
cophenetic_corr_single <- cophenetic(dendrogram_single)
cophenetic_corr_centroid <- cophenetic(dendrogram_centroid)
cophenetic_corr_ward <- cophenetic(dendrogram_ward)
cat("Coeficiente de correlación de Cophenetic para el método Single:", cor(dist(mall_customers[, c("Age", "Annual.Income", "Spending.Score")]), cophenetic_corr_single), "\n")
cat("Coeficiente de correlación de Cophenetic para el método Centroid:", cor(dist(mall_customers[, c("Age", "Annual.Income", "Spending.Score")]), cophenetic_corr_centroid), "\n")
cat("Coeficiente de correlación de Cophenetic para el método Ward:", cor(dist(mall_customers[, c("Age", "Annual.Income", "Spending.Score")]), cophenetic_corr_ward), "\n")

Observamos que el coeficiente más cercano a 1 es el que corresponde al método Centroid, por lo que lo escogemos antes que los otros dos

In [ ]:
groups <- cutree(dendrogram_centroid, k = 5)
mall_customers$dendrogram <- groups

Graficamos los resultados

In [ ]:
ggplot(mall_customers, aes(x = Annual.Income, y = Spending.Score, color = factor(groups))) +
  geom_point() +
  labs(title = "Segmentación de clientes utilizando Agrupamiento Jerárquico", x = "Ingresos Anuales", y = "Puntuación de Gasto") +
  theme_minimal()

Visualizamos los resultados de forma numérica

In [ ]:
table(mall_customers$dendrogram)
prop.table(table(mall_customers$dendrogram))

### **Repetimos las tareas por género**

Separamos los datos por género

In [ ]:
mall_customers <- read.csv("mallCustomers.csv")
mall_customers_male <- subset(mall_customers, Genre == "Male")
mall_customers_female <- subset(mall_customers, Genre == "Female")

##### **k-means Género Hombre**

Determinamos el número óptimo de centroides usando el método del codo

In [ ]:

set.seed(123)
wcss_male <- vector()
for (i in 1:10) {
  kmeans_model_male <- kmeans(mall_customers_male[, c("Age", "Annual.Income", "Spending.Score")], centers = i, nstart = 10)
  wcss_male[i] <- kmeans_model_male$tot.withinss
}
plot(1:10, wcss_male, type = 'b', main = "Método del Codo", xlab = "Número de Clústeres (k)", ylab = "WCSS")

El número de centroides será 5

In [ ]:
kmeans_male <- kmeans(mall_customers_male[, c("Age", "Annual.Income", "Spending.Score")], centers = 5, nstart = 10)
mall_customers_male$cluster <- kmeans_male$cluster

Graficamos los resultados

In [ ]:
ggplot(mall_customers_male, aes(x = Annual.Income, y = Spending.Score, color = factor(cluster))) +
  geom_point() +
  geom_point(data = kmeans_male$centers, aes(x = Annual.Income, y = Spending.Score), color = 'black', size = 3, shape = 16) +
  labs(title = "Segmentación de clientes masculinos utilizando k-means", x = "Ingresos Anuales", y = "Puntuación de Gasto") +
  theme_minimal()

Visualizamos los resultados de forma numérica

In [ ]:
table(mall_customers_male$cluster)
prop.table(table(mall_customers_male$cluster))

##### **k-means Género Mujer**

Determinamos el número óptimo de centroides usando el método del codo

In [ ]:
set.seed(123)
wcss_female <- vector()
for (i in 1:10) {
  kmeans_model_female <- kmeans(mall_customers_female[, c("Age", "Annual.Income", "Spending.Score")], centers = i, nstart = 10)
  wcss_female[i] <- kmeans_model_female$tot.withinss
}
plot(1:10, wcss_female, type = 'b', main = "Método del Codo", xlab = "Número de Clústeres (k)", ylab = "WCSS")

El número de centroides será 5

In [ ]:
kmeans_female <- kmeans(mall_customers_female[, c("Age", "Annual.Income", "Spending.Score")], centers = 5, nstart = 10)
mall_customers_female$cluster <- kmeans_female$cluster

Graficamos los resultados

In [ ]:
ggplot(mall_customers_female, aes(x = Annual.Income, y = Spending.Score, color = factor(cluster))) +
  geom_point() +
  geom_point(data = kmeans_female$centers, aes(x = Annual.Income, y = Spending.Score), color = 'black', size = 3, shape = 16) +
  labs(title = "Segmentación de clientes femeninos utilizando k-means", x = "Ingresos Anuales", y = "Puntuación de Gasto") +
  theme_minimal()

Visualizamos los resultados de forma numérica

In [ ]:
table(mall_customers_female$cluster)
prop.table(table(mall_customers_female$cluster))

##### **Dendrogramas Jerárquicos Género Hombre**

##### **Single**

In [ ]:
dendrogram_male_single <- hclust(dist(mall_customers_male[, c("Age", "Annual.Income", "Spending.Score")]), method = "single")

Graficamos los resultados

In [ ]:
plot(dendrogram_male_single, main = "Single", xlab = "", ylab = "Distance")

##### **Centroid**

In [ ]:
dendrogram_male_centroid <- hclust(dist(mall_customers_male[, c("Age", "Annual.Income", "Spending.Score")]), method = "centroid")

Graficamos los resultados

In [ ]:
plot(dendrogram_male_centroid, main = "Centroid", xlab = "", ylab = "Distance")

##### **Ward**

In [ ]:
dendrogram_male_ward <- hclust(dist(mall_customers_male[, c("Age", "Annual.Income", "Spending.Score")]), method = "ward.D")

Graficamos los resultados

In [ ]:
plot(dendrogram_male_ward, main = "Ward", xlab = "", ylab = "Distance")

##### **Evaluación de los dendrogramas**

Comparamos los dendrogramas y nos quedamos con el que preserva mejor las distancias originales mediante el coeficiente de correlación de Cophenetic

In [ ]:
cophenetic_corr_single_male <- cophenetic(dendrogram_male_single)
cophenetic_corr_centroid_male <- cophenetic(dendrogram_male_centroid)
cophenetic_corr_ward_male <- cophenetic(dendrogram_male_ward)
cat("Coeficiente de correlación de Cophenetic para el método Single:", cor(dist(mall_customers_male[, c("Age", "Annual.Income", "Spending.Score")]), cophenetic_corr_single_male), "\n")
cat("Coeficiente de correlación de Cophenetic para el método Centroid:", cor(dist(mall_customers_male[, c("Age", "Annual.Income", "Spending.Score")]), cophenetic_corr_centroid_male), "\n")
cat("Coeficiente de correlación de Cophenetic para el método Ward:", cor(dist(mall_customers_male[, c("Age", "Annual.Income", "Spending.Score")]), cophenetic_corr_ward_male), "\n")

Al igual que con el conjunto de datos completo, el dendrograma que preserva mejor las distancias originales es el método Centroid

##### **Dendrogramas Jerárquicos Género Mujer**

##### **Single**

In [ ]:
dendrogram_female_single <- hclust(dist(mall_customers_female[, c("Age", "Annual.Income", "Spending.Score")]), method = "single")

Graficamos los resultados

In [ ]:
plot(dendrogram_female_single, main = "Single", xlab = "", ylab = "Distance")

##### **Centroid**

In [ ]:
dendrogram_female_centroid <- hclust(dist(mall_customers_female[, c("Age", "Annual.Income", "Spending.Score")]), method = "centroid")

Graficamos los resultados

In [ ]:
plot(dendrogram_female_centroid, main = "Centroid", xlab = "", ylab = "Distance")

##### **Ward**

In [ ]:
dendrogram_female_ward <- hclust(dist(mall_customers_female[, c("Age", "Annual.Income", "Spending.Score")]), method = "ward.D")

Graficamos los resultados

In [ ]:
plot(dendrogram_female_ward, main = "Ward", xlab = "", ylab = "Distance")

##### **Evaluación de los dendrogramas**

Comparamos los dendrogramas y nos quedamos con el que preserva mejor las distancias originales mediante el coeficiente de correlación de Cophenetic

In [ ]:
cophenetic_corr_single_female <- cophenetic(dendrogram_female_single)
cophenetic_corr_centroid_female <- cophenetic(dendrogram_female_centroid)
cophenetic_corr_ward_female <- cophenetic(dendrogram_female_ward)
cat("Coeficiente de correlación de Cophenetic para el método Single:", cor(dist(mall_customers_female[, c("Age", "Annual.Income", "Spending.Score")]), cophenetic_corr_single_female), "\n")
cat("Coeficiente de correlación de Cophenetic para el método Centroid:", cor(dist(mall_customers_female[, c("Age", "Annual.Income", "Spending.Score")]), cophenetic_corr_centroid_female), "\n")
cat("Coeficiente de correlación de Cophenetic para el método Ward:", cor(dist(mall_customers_female[, c("Age", "Annual.Income", "Spending.Score")]), cophenetic_corr_ward_female), "\n")

Al igual que con el conjunto de datos completo y separado por el género hombre, el dendrograma que preserva mejor las distancias originales es el método Centroid

### **Diferencias entre agrupaciones por género**

In [ ]:
ggplot(mall_customers_male, aes(x = Annual.Income, y = Spending.Score, color = factor(cluster))) +
  geom_point() +
  geom_point(data = kmeans_male$centers, aes(x = Annual.Income, y = Spending.Score), color = 'black', size = 3, shape = 16) +
  labs(title = "Segmentación de clientes masculinos utilizando k-means", x = "Ingresos Anuales", y = "Puntuación de Gasto") +
  theme_minimal()

In [ ]:
table(mall_customers_male$cluster)
prop.table(table(mall_customers_male$cluster))

In [ ]:
ggplot(mall_customers_female, aes(x = Annual.Income, y = Spending.Score, color = factor(cluster))) +
  geom_point() +
  geom_point(data = kmeans_female$centers, aes(x = Annual.Income, y = Spending.Score), color = 'black', size = 3, shape = 16) +
  labs(title = "Segmentación de clientes femeninos utilizando k-means", x = "Ingresos Anuales", y = "Puntuación de Gasto") +
  theme_minimal()

In [ ]:
table(mall_customers_female$cluster)
prop.table(table(mall_customers_female$cluster))

Si comparamos los resultados arrojados por las agrupaciones, observamos que no hay una diferencia realmente notable entre los distintos grupos generados por ambas agrupaciones.

Sin embargo, si nos fijamos en los centroides de ambas agrupaciones, nos podemos dar cuenta que los centroides de las agrupaciones del conjunto de clientas femeninas tienen un valor mayor en el eje y respecto a los clientes masculinos. Esto significa que, en promedio, las clientes femeninas tienen una puntuación de gasto mayor que los clientes masculinos.

Por otro lado, si nos fijamos en los resultados numéricos, observamos que los porcentajes de las agrupaciones del género masculino están bastante repartidas uniformemente, siendo el grupo 2 el mayoritario, mientras que el grupo mayoritario de clientas femeninas es el 1, con un 41% de las mismas.